# 날짜 데이터

- how to change the system time in Colab
  - Colab uses UTC (Coordinated Universal Time) time as the default system time.
  - To change the system time in Colab, run the following code snippet.
  - you will nedd to restart the runtime for Colab to recognize it.

In [2]:
import time
time.time(), time.localtime(time.time())

(1668144245.7356427,
 time.struct_time(tm_year=2022, tm_mon=11, tm_mday=11, tm_hour=14, tm_min=24, tm_sec=5, tm_wday=4, tm_yday=315, tm_isdst=0))

In [3]:
# change the system time for Korea in Colab
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Seoul /etc/localtime
!date

Fri Nov 11 14:24:31 KST 2022


In [4]:
import time
time.time()   # expressed in seconds since the epoch

1668144286.7865248

In [5]:
# convert it to time (format)
time.localtime(time.time())

time.struct_time(tm_year=2022, tm_mon=11, tm_mday=11, tm_hour=14, tm_min=24, tm_sec=47, tm_wday=4, tm_yday=315, tm_isdst=0)

- now, we are in Korea time zone.

In [6]:
from datetime import datetime, timedelta, timezone

now = datetime.now()          # current date and time (colab or jupyter?) - default is UTC time
now_utc = datetime.utcnow()   # current UTC date and time (영국 그리니치 천문대, 경도 0)
print ('time in Korea:{}'.format(now))
print ('time in UTC:  {}  '.format(now_utc))

time in Korea:2022-11-11 14:24:54.642139
time in UTC:  2022-11-11 05:24:54.642193  


In [7]:
type(now)

datetime.datetime

In [8]:
dt_kst = datetime.utcnow() + timedelta(hours=9)
print(dt_kst)

2022-11-11 14:25:42.989306


## naive datetime and aware datetime
- naive datetime (timezone 정보가 없음)
- aware datetime (timezone 정보가 있음)
- 함께 계산이 안됨 (항상 aware datetime 사용 권장)

In [9]:
now_naive = datetime.now()  # naive
now_naive

datetime.datetime(2022, 11, 11, 14, 25, 57, 862367)

In [10]:
# convert nive to aware instance (with Korea time zone)
KST = timezone(timedelta(hours=9))   # time zone: 한국은 UTC 와 9 시간 차이
now_aware = now_naive.astimezone(KST)
now_aware                   # aware

datetime.datetime(2022, 11, 11, 14, 25, 57, 862367, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400)))

In [11]:
try:
    print(now_aware - now_naive)
except:
    print("Error: naive datetime과 aware datetime을 서로 섞어서 쓰지 못한다.")

Error: naive datetime과 aware datetime을 서로 섞어서 쓰지 못한다.


In [12]:
time1 = datetime(2019, 1, 1, 0, 0, 0, tzinfo=KST)  # another aware
time1

datetime.datetime(2019, 1, 1, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400)))

In [13]:
now_aware - time1    # aware datetimes can be subtracted.

datetime.timedelta(days=1410, seconds=51957, microseconds=862367)

- extract detailed information

In [14]:
# 정보 추출하기
KST = timezone(timedelta(hours=9)) 
now = datetime.now()
now_kr = now.astimezone(KST)
print(now)
print(now_kr)
print (now_kr.year)
print (now_kr.month)
print (now_kr.day)
print(now_kr.hour)
print(now_kr.minute)
print(now_kr.second)
print(now_kr.microsecond)
print(now_kr.weekday())      #  Monday is 0

2022-11-11 14:27:12.990028
2022-11-11 14:27:12.990028+09:00
2022
11
11
14
27
12
990028
4


In [15]:
# 날짜 정보 다듬기

KST = timezone(timedelta(hours=9)) 
now_kr = datetime.now().astimezone(KST)
print (str(now_kr.month) + "/" + str(now_kr.day) + "/" + str(now_kr.year))

11/11/2022


In [16]:
# 시간 정보 다듬기

print (str(now_kr.hour) + ":" + str(now_kr.minute) + ":" + str(now_kr.second))

14:27:35


- time zone 확인

In [17]:
import pytz     # time zone package

for tz in pytz.all_timezones:
    if 'Seoul' in tz:
         print(tz)

Asia/Seoul


In [18]:
KST = pytz.timezone('Asia/Seoul')
UTC = pytz.utc
date_utc = datetime.now(UTC)
date_kor = datetime.now(KST)
print(date_utc)
print(date_kor)

2022-11-11 05:28:29.138676+00:00
2022-11-11 14:28:29.138744+09:00


# 영업일 캘린더 (KRX)

* 한국거래소 홈페이지에서 휴장일을 xls로 받을 수 있다
* 휴장일 데이터로 부터 개장일(market days)를 생성
* 해외 시장(CME, EUREX)등은  pandas-market-calendars를 활용

<img width="320" src="http://i.imgur.com/ClIQff8.jpg" >

#### http://fb.com/financedata , http://financedata.github.io
<!-- TEASER_END -->

In [19]:
# you need newer version of xlrd to read 'xlxs' file
!pip install xlrd=='1.2.0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import pandas as pd
import xlrd
pd.__version__, xlrd.__version__

('1.3.5', '1.2.0')

In [21]:
import pandas as pd

df_hdays = pd.read_excel('https://goo.gl/JGse1n')   # 휴장일 정보 (.xls file)
df_hdays

,일자 및 요일,요일구분,비고
0,2019-01-01,화요일,신정
1,2019-02-04,월요일,설날
2,2019-02-05,화요일,설날
3,2019-02-06,수요일,설날
4,2019-03-01,금요일,삼일절
5,2019-05-01,수요일,근로자의날
6,2019-05-06,월요일,어린이날(대체휴일)
7,2019-06-06,목요일,현충일
8,2019-08-15,목요일,광복절
9,2019-09-12,목요일,추석


날짜 데이터에 요일이 함께 들어있어 가공이 필요하다.


In [22]:
# hdays = df_hdays['일자 및 요일'].str.extract('(\d{4}-\d{2}-\d{2})', expand=False)
hdays = df_hdays['일자 및 요일']
hdays

0     2019-01-01
1     2019-02-04
2     2019-02-05
3     2019-02-06
4     2019-03-01
5     2019-05-01
6     2019-05-06
7     2019-06-06
8     2019-08-15
9     2019-09-12
10    2019-09-13
11    2019-10-03
12    2019-10-09
13    2019-12-25
14    2019-12-31
Name: 일자 및 요일, dtype: object

In [23]:
hdays = pd.to_datetime(hdays)    # string -> datetime type conversion
hdays.name = '날짜'
hdays                             # series

0    2019-01-01
1    2019-02-04
2    2019-02-05
3    2019-02-06
4    2019-03-01
5    2019-05-01
6    2019-05-06
7    2019-06-06
8    2019-08-15
9    2019-09-12
10   2019-09-13
11   2019-10-03
12   2019-10-09
13   2019-12-25
14   2019-12-31
Name: 날짜, dtype: datetime64[ns]

# 개장일(market day)

2019년 중에 개장일(market day)목록을 만들어 보자. 우선 pandas.date_range() 함수를 사용하여 1월 1일~ 12월 31까지 날짜를 생성한다. (365일)

In [24]:
mdays = pd.date_range('2019-01-01', '2019-12-31')
mdays

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

이때 freq 를 'B'로 지정(business day)하면, 주말(토,일)은 제외된다. (260일)

날짜 생성에 대해서는 다음 URL을 참고 하자. 
https://pandas.pydata.org/pandas-docs/stable/timeseries.html

특히, 간격 별칭(Offset Aliases)에 대해서는 아래 표에 정리되어 있다.

https://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases

In [30]:
mdays = pd.date_range('2019-01-01', '2019-12-31', freq='B') # business day
mdays, len(mdays)

(DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
                '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10',
                '2019-01-11', '2019-01-14',
                ...
                '2019-12-18', '2019-12-19', '2019-12-20', '2019-12-23',
                '2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27',
                '2019-12-30', '2019-12-31'],
               dtype='datetime64[ns]', length=261, freq='B'), 261)

여기서 휴일을 제외(df.drop)하면, 모든 영업일(market day)가 만들어진다. (244일)

In [31]:
mdays = mdays.drop(hdays)
mdays, len(mdays)

(DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
                '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
                '2019-01-14', '2019-01-15',
                ...
                '2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19',
                '2019-12-20', '2019-12-23', '2019-12-24', '2019-12-26',
                '2019-12-27', '2019-12-30'],
               dtype='datetime64[ns]', length=246, freq=None), 246)

# DataFrame에 휴장일 적용하기

DataFrame에서 개장일(market day)만을 추출해해야 하는 경우를 살펴보자.

다음과 같은 데이터를 가정해 보자. (5월 1일\~5월 30일, 1~30까지 values로 할당)

In [32]:
data = {'values': range(1,32)}; data

{'values': range(1, 32)}

In [33]:
df_sample = pd.DataFrame(data, index=pd.date_range('2019-05-01', '2019-05-31'))
df_sample.head()

,values
2019-05-01,1
2019-05-02,2
2019-05-03,3
2019-05-04,4
2019-05-05,5


위 데이터 중에서 개장일만 추출하고자 한다. DateTimeIndex를 가진 두 개의 DataFrame의 교집합을 구하는 간단한 방법으로 pd.merge()을 쓸 수 있다.

In [34]:
# 개장일을 index로 갖는 DataFrame
df_mdays = pd.DataFrame(index=mdays)
df_mdays.head()

""
2019-01-02
2019-01-03
2019-01-04
2019-01-07
2019-01-08


In [35]:
# 두 DataFrame (df_sample, df_mdays)의 인덱스를 기준으로 합친다(merge)
df = pd.merge(df_sample, df_mdays, right_index=True, left_index=True)
            # join is done on columns or indexes
            # use the index from the left(right) df as the join key(s)
            # merging: 공통된 값을 기준으로 합치기
            # concatenate: 단순 합치기
df.head(3).append(df.tail(3))

,values
2019-05-02,2
2019-05-03,3
2019-05-07,7
2019-05-29,29
2019-05-30,30
2019-05-31,31


이 경우, 두 인덱스의 공동부분(intersection)만 남으므로 개장일만 남게 된다.

# CME, EUREX 휴장일

영업일 캘린더 메뉴에서 한국거래소(KRX)외에 CME, EUREX와 캘린더도 제공하며, 엑셀로 다운로드 한 2017년 데이터는 다음과 같다.
- CME(Chicago Mercantile Exchange): 세계 최대 선물 거래소

* hdays-2017-CME.xls ( https://goo.gl/cvnhy2 ) 
* hdays-2017-EUREX.xls ( https://goo.gl/s2PgBx ) 

In [36]:
pd.read_excel('https://goo.gl/cvnhy2')

,구분,일자,요일,시장,비고
0,휴장,2017/01/02,월,CME,New Year' Day
1,휴장,2017/04/14,금,CME,Good Friday
2,단축종료(03:15),2017/11/24,금,CME,추수감사절 다음날
3,휴장,2017/12/25,월,CME,Christmas Day


In [37]:
pd.read_excel('https://goo.gl/s2PgBx')

,구분,일자,요일,시장,비고
0,휴장,2017/04/14,금,EUREX,Good Friday
1,휴장,2017/04/17,월,EUREX,Easter Monday
2,휴장,2017/05/01,월,EUREX,Labour Day
3,휴장,2017/12/25,월,EUREX,Christmas Day
4,휴장,2017/12/26,화,EUREX,Boxing Day


# pandas-market-calendars
- 해외 시장의 개장일(market day)을 관리하기 위해 위 엑셀 데이터를 사용하기 보다는 pandas-market-calendars 라이브러를 권한다. http://pandas-market-calendars.readthedocs.io
- NYSE, CME, EUREX 등 시장에 대한 개장일과 개장 시간등을 관리할 수 있다.
- 2017 https://fb.com/financedata